In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from pathlib import Path
#pd.set_option("display.max_rows", 50000000)
pd.set_option("display.max_columns", 500)

In [50]:
df = pd.read_csv("data/combo.csv", \
                 dtype={'npi':object,
                        'lastname':object,
                        'firstname':object,
                        'credentials':object,
                        'gender':object,
                        'address':object,
                        'city':object,
                        'zip':object,
                        'state':object,
                        'provider_type':object,
                        'place_of_service':object,
                        'hcpcs_code':object,
                        'hcpcs_description':object,
                        'EXCLTYPE':object,
                        'year':object})

In [51]:
df.columns

Index(['npi', 'lastname', 'firstname', 'credentials', 'gender', 'address',
       'city', 'zip', 'state', 'provider_type', 'place_of_service',
       'hcpcs_code', 'hcpcs_description', 'line_srvc_cnt', 'bene_unique_cnt',
       'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt',
       'average_Medicare_standard_amt', 'Total_submitted_chrg_amt',
       'Total_Medicare_payment_amt', 'EXCLTYPE', 'year', 'Target'],
      dtype='object')

In [52]:
df['Total_Medicare_std_payment_amt'] = df.line_srvc_cnt * df.average_Medicare_standard_amt

In [53]:
df['ratio_pay/submit'] = df.Total_Medicare_std_payment_amt/ df.Total_submitted_chrg_amt

In [54]:
df['Total_Net'] = df.Total_submitted_chrg_amt - df.Total_Medicare_std_payment_amt

In [55]:
df['Total_Medicare_allowed_amt'] = df.line_srvc_cnt * df.average_Medicare_allowed_amt

In [56]:
df['ratio_pay/allowed'] = df.Total_Medicare_std_payment_amt/ df.Total_Medicare_allowed_amt

In [57]:
df['Net_allow_pay'] = df['Total_Medicare_allowed_amt'] - df['Total_Medicare_std_payment_amt']

In [58]:
(df.line_srvc_cnt == df.bene_day_srvc_cnt).sum()

1483371

In [59]:
df.shape

(1511529, 31)

In [61]:
df1 = df.drop(["lastname", "firstname", "address", "credentials", "city", "zip",\
         "provider_type", "place_of_service", "hcpcs_description","bene_day_srvc_cnt",\
         'average_Medicare_payment_amt','Total_Medicare_payment_amt'], axis=1)

In [62]:
df1.head()

,npi,gender,state,hcpcs_code,line_srvc_cnt,bene_unique_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,EXCLTYPE,year,Target,Total_Medicare_std_payment_amt,ratio_pay/submit,Total_Net,Total_Medicare_allowed_amt,ratio_pay/allowed,Net_allow_pay
0,1003000126,M,MD,99223,114.0,110,206.061579,923.991228,160.026316,105335.000000,None,None,0,18243.00,0.173190,87092.000000,23491.020000,0.776595,5248.020000
1,1003000126,M,MD,99232,627.0,233,72.622456,326.245614,57.047895,204556.000003,None,None,0,35769.03,0.174862,168786.970003,45534.280000,0.785541,9765.250000
2,1003000126,M,MD,99233,207.0,127,106.653913,461.072464,83.196039,95442.000000,None,None,0,17221.58,0.180440,78220.420000,22077.359999,0.780056,4855.779999
3,1003000126,M,MD,99291,36.0,23,224.170000,1400.000000,177.656667,50400.000000,None,None,0,6395.64,0.126898,44004.360000,8070.120000,0.792509,1674.480000
4,1003000142,M,OH,99213,129.0,89,70.930000,109.000000,53.268372,14061.000000,None,None,0,6871.62,0.488701,7189.380000,9149.970000,0.750999,2278.350000


In [74]:
cols = ['XX','AA', 'AE','AP','AS','GU','MP','PR','VI','ZZ']
len(cols)

df1 = df1[~df1.state.isin(cols)]

In [76]:
df1.head()

,npi,gender,state,hcpcs_code,line_srvc_cnt,bene_unique_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_standard_amt,Total_submitted_chrg_amt,EXCLTYPE,year,Target,Total_Medicare_std_payment_amt,ratio_pay/submit,Total_Net,Total_Medicare_allowed_amt,ratio_pay/allowed,Net_allow_pay
0,1003000126,M,MD,99223,114.0,110,206.061579,923.991228,160.026316,105335.000000,None,None,0,18243.00,0.173190,87092.000000,23491.020000,0.776595,5248.020000
1,1003000126,M,MD,99232,627.0,233,72.622456,326.245614,57.047895,204556.000003,None,None,0,35769.03,0.174862,168786.970003,45534.280000,0.785541,9765.250000
2,1003000126,M,MD,99233,207.0,127,106.653913,461.072464,83.196039,95442.000000,None,None,0,17221.58,0.180440,78220.420000,22077.359999,0.780056,4855.779999
3,1003000126,M,MD,99291,36.0,23,224.170000,1400.000000,177.656667,50400.000000,None,None,0,6395.64,0.126898,44004.360000,8070.120000,0.792509,1674.480000
4,1003000142,M,OH,99213,129.0,89,70.930000,109.000000,53.268372,14061.000000,None,None,0,6871.62,0.488701,7189.380000,9149.970000,0.750999,2278.350000


In [98]:
# Excluded unique NPI
ex_u = df1[df1.Target==1]['npi'].nunique()
ex_u

140

In [99]:
# Non-excluded unique NPI
nex_u = df1[df1.Target==0]['npi'].nunique()
nex_u

648637

In [100]:
tot = ex_u + nex_u
tot

648777

In [106]:
1/(ex_u/tot)

4634.121428571429

In [107]:
1/(nex_u/tot)

1.0002158372094099

In [108]:
df1['weight'] = np.where(df1.Target==1, 1/(ex_u/tot), 1/(nex_u/tot))

In [117]:
df1.weight.value_counts()

1.000216       1505687
4634.121429        285
Name: weight, dtype: int64

In [122]:
df1.columns

Index(['npi', 'gender', 'state', 'hcpcs_code', 'line_srvc_cnt',
       'bene_unique_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_standard_amt',
       'Total_submitted_chrg_amt', 'EXCLTYPE', 'year', 'Target',
       'Total_Medicare_std_payment_amt', 'ratio_pay/submit', 'Total_Net',
       'Total_Medicare_allowed_amt', 'ratio_pay/allowed', 'Net_allow_pay',
       'weight'],
      dtype='object')

In [123]:
df_wide = df.pivot_table(index='npi', columns='hcpcs_code', 
         values=['line_srvc_cnt', 'bene_unique_cnt', 'average_Medicare_allowed_amt', \
                 'average_submitted_chrg_amt', 'average_Medicare_standard_amt',\
                 'Total_submitted_chrg_amt', 'Total_Medicare_std_payment_amt',
                 'Total_Medicare_allowed_amt',\
                 'ratio_pay/submit', 'ratio_pay/allowed',\
                 'Total_Net','Net_allow_pay'])

In [126]:
df_wide.head()

Net_allow_pay                                                    \
hcpcs_code         66984    99213     99214    99223    99232        99233   
npi                                                                          
1003000126           NaN      NaN       NaN  5248.02  9765.25  4855.779999   
1003000142           NaN  2278.35  4497.905      NaN      NaN          NaN   
1003000407           NaN      NaN       NaN  2897.41  3512.85  1602.480000   
1003000423           NaN   382.41       NaN      NaN      NaN          NaN   
1003000480           NaN   399.93       NaN      NaN      NaN          NaN   

                                      Total_Medicare_allowed_amt           \
hcpcs_code 99284 99285    99291 A0427                      66984    99213   
npi                                                                         
1003000126   NaN   NaN  1674.48   NaN                        NaN      NaN   
1003000142   NaN   NaN      NaN   NaN                        NaN  9149.97   
1003000407   NaN   NaN      NaN   NaN                        NaN      NaN   
1003000423   NaN   NaN      NaN   NaN                        NaN  1489.53   
1003000480   NaN   NaN      NaN   NaN                        NaN  1301.50   

                                                                             \
hcpcs_code     99214     99223     99232         99233 99284 99285    99291   
npi                                                                           
1003000126       NaN  23491.02  45534.28  22077.359999   NaN   NaN  8070.12   
1003000142  18181.53       NaN       NaN           NaN   NaN   NaN      NaN   
1003000407       NaN  14840.33  17327.90   7901.720000   NaN   NaN      NaN   
1003000423       NaN       NaN       NaN           NaN   NaN   NaN      NaN   
1003000480       NaN       NaN       NaN           NaN   NaN   NaN      NaN   

                 Total_Medicare_std_payment_amt                                \
hcpcs_code A0427                          66984    99213      99214     99223   
npi                                                                             
1003000126   NaN                            NaN      NaN        NaN  18243.00   
1003000142   NaN                            NaN  6871.62  13683.625       NaN   
1003000407   NaN                            NaN      NaN        NaN  11942.92   
1003000423   NaN                            NaN  1107.12        NaN       NaN   
1003000480   NaN                            NaN   901.57        NaN       NaN   

                                                          Total_Net           \
hcpcs_code     99232     99233 99284 99285    99291 A0427     66984    99213   
npi                                                                            
1003000126  35769.03  17221.58   NaN   NaN  6395.64   NaN       NaN      NaN   
1003000142       NaN       NaN   NaN   NaN      NaN   NaN       NaN  7189.38   
1003000407  13815.05   6299.24   NaN   NaN      NaN   NaN       NaN      NaN   
1003000423       NaN       NaN   NaN   NaN      NaN   NaN       NaN   782.88   
1003000480       NaN       NaN   NaN   NaN      NaN   NaN       NaN  4113.43   

                                                                      \
hcpcs_code      99214     99223          99232     99233 99284 99285   
npi                                                                    
1003000126        NaN  87092.00  168786.970003  78220.42   NaN   NaN   
1003000142  13931.375       NaN            NaN       NaN   NaN   NaN   
1003000407        NaN   6068.08   22334.950000   7000.76   NaN   NaN   
1003000423        NaN       NaN            NaN       NaN   NaN   NaN   
1003000480        NaN       NaN            NaN       NaN   NaN   NaN   

                           Total_submitted_chrg_amt                    \
hcpcs_code     99291 A0427                    66984    99213    99214   
npi                                                                     
1003000126  44004.36   NaN                      Na

In [127]:
def imputezeros(df_input):
    df = df_input.copy()
    for col in df.columns:
        df[col] = np.where(df[col].isnull(), int(0), df[col])
    return(df)

In [128]:
df_wide = imputezeros(df_wide)

In [130]:
df_wide.head(20)

Net_allow_pay                                                  \
hcpcs_code         66984         99213      99214         99223    99232   
npi                                                                        
1003000126           0.0      0.000000      0.000   5248.020000  9765.25   
1003000142           0.0   2278.350000   4497.905      0.000000     0.00   
1003000407           0.0      0.000000      0.000   2897.410000  3512.85   
1003000423           0.0    382.410000      0.000      0.000000     0.00   
1003000480           0.0    399.930000      0.000      0.000000     0.00   
1003000522           0.0   8636.970000  18682.550   1626.440000  1344.01   
1003000530           0.0   3458.260000  26501.800      0.000000     0.00   
1003000597           0.0   4606.050000   6500.450      0.000000   236.16   
1003000738           0.0    460.500000      0.000      0.000000     0.00   
1003000902           0.0   9860.380000   5122.940      0.000000     0.00   
1003000936           0.0    396.530000  12252.590   2115.380000   552.98   
1003001017           0.0  10664.080000   1049.860      0.000000     0.00   
1003001256           0.0    748.510000   1235.020      0.000000     0.00   
1003001363           0.0  97373.039999   5398.590      0.000000     0.00   
1003001371           0.0      0.000000   3741.130    609.920000  4107.71   
1003001462           0.0    195.840000      0.000      0.000000     0.00   
1003001587           0.0      0.000000      0.000      0.000000     0.00   
1003001645           0.0    809.320000   3596.080  13937.729999   451.36   
1003001785           0.0    686.530000   3869.500      0.000000     0.00   
1003001884           0.0   2251.570000   4703.130      0.000000     0.00   

                                                          \
hcpcs_code         99233    99284   99285    99291 A0427   
npi                                                        
1003000126   4855.779999     0.00    0.00  1674.48   0.0   
1003000142      0.000000     0.00    0.00     0.00   0.0   
1003000407   1602.480000     0.00    0.00     0.00   0.0   
1003000423      0.000000     0.00    0.00     0.00   0.0   
1003000480      0.000000     0.00    0.00     0.00   0.0   
1003000522    395.930000     0.00    0.00     0.00   0.0   
1003000530      0.000000     0.00    0.00     0.00   0.0   
1003000597      0.000000     0.00    0.00     0.00   0.0   
1003000738      0.000000     0.00    0.00     0.00   0.0   
1003000902      0.000000     0.00    0.00     0.00   0.0   
1003000936      0.000000     0.00    0.00     0.00   0.0   
1003001017      0.000000     0.00    0.00     0.00   0.0   
1003001256      0.000000     0.00    0.00     0.00   0.0   
1003001363      0.000000     0.00    0.00     0.00   0.0   
1003001371      0.000000     0.00    0.00     0.00   0.0   
1003001462      0.000000     0.00    0.00     0.00   0.0   
1003001587      0.000000  1323.17  998.42     0.00   0.0   
1003001645  11497.540000     0.00    0.00     0.00   0.0   
1003001785      0.000000     0.00    0.00     0.00   0.0   
1003001884      0.000000     0.00    0.00     0.00   0.0   

           Total_Medicare_allowed_amt                                     \
hcpcs_code                      66984          99213     99214     99223   
npi                                                                        
1003000126                        0.0       0.000000      0.00  23491.02   
1003000142                        0.0    9149.970000  18181.53      0.00   
1003000407                        0.0       0.000000      0.00  14840.33   
1003000423                        0.0    1489.530000      0.00      0.00   
1003000480                        0.0    1301.500000      0.00      0.00   
1003000522                        0.0   26341.200000  53521.93   5786.20   
1003000530                        0.0    8788.640000  69374.48      0.00   
1003000597                        0.0   22195.200000  26094.15      0.00   
1003000738                        0.0    1558